In [15]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam

import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
import time

In [16]:
#
batch_size = 100
nb_classes = 10
nb_epoch = 10
log_filepath = './log'

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = (32, 64, 1024)
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [17]:
old_session = KTF.get_session()

In [36]:
with tf.Graph().as_default():
    session = tf.Session('')
    KTF.set_session(session)
    KTF.set_learning_phase(1)
    
    ################## Model ###########################
    model = Sequential()
    
    with tf.name_scope("inference") as scope:
        #Layer 1
        model.add(Convolution2D(nb_filters[0], kernel_size[0], kernel_size[1],
                                border_mode='valid', input_shape=input_shape, name='cnv1a'))
        model.add(Activation('relu', name='relu1a'))
        model.add(Convolution2D(nb_filters[0], kernel_size[0], kernel_size[1], name='cnv1b'))
        model.add(Activation('relu', name='relu1b'))
        model.add(MaxPooling2D(pool_size=pool_size, name='maxpool1'))
        #model.add(Dropout(0.25, name='dropout1'))
        #Layer 2
        model.add(Convolution2D(nb_filters[1], kernel_size[0], kernel_size[1], name='cnv2a'))
        model.add(Activation('relu', name='relu2a'))
        model.add(Convolution2D(nb_filters[1], kernel_size[0], kernel_size[1], name='cnv2b'))
        model.add(Activation('relu', name='relu2b'))
        model.add(MaxPooling2D(pool_size=pool_size, name='maxpool2'))
        #model.add(Dropout(0.25, name='dropout2'))
        #Layer 3
        model.add(Flatten(name='flatten3'))
        model.add(Dense(nb_filters[2], name='dense3'))
        model.add(Activation('relu', name='relu3'))
        #model.add(Dropout(0.5, name='dropout3'))
        #Layer 4
        model.add(Dense(nb_classes, name='dense4'))
        model.add(Activation('softmax', name='softmax'))
    model.summary()
    ####################################################

    #Optimizer
    adam = Adam(lr=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    
    #Call back
    tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=1)
    cbks = [tb_cb]
    
    #Fit
    start = time.time()
    history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch = nb_epoch, 
                        verbose=2, callbacks=cbks, validation_data=(X_test, Y_test), shuffle=True)
    elapsed_time = time.time() - start
    
    score = model.evaluate(X_test, Y_test, verbose=0)
    #Output
    print(("finish_time: {0}".format(int(elapsed_time))) + " [sec]")
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    
#Save model
model.save('my_model2.h5')
    
KTF.set_session(old_session)    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cnv1a (Convolution2D)            (None, 26, 26, 32)    320         convolution2d_input_22[0][0]     
____________________________________________________________________________________________________
relu1a (Activation)              (None, 26, 26, 32)    0           cnv1a[0][0]                      
____________________________________________________________________________________________________
cnv1b (Convolution2D)            (None, 24, 24, 32)    9248        relu1a[0][0]                     
____________________________________________________________________________________________________
relu1b (Activation)              (None, 24, 24, 32)    0           cnv1b[0][0]                      
___________________________________________________________________________________________

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,26,26]
	 [[Node: inference/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_inference/convolution2d_input_22_0/_73, inference/cnv1a_W/read)]]
	 [[Node: inference/add_1/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_58_inference/add_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'inference/Conv2D', defined at:
  File "C:\Users\shuto05\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\shuto05\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\shuto05\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.l